# Data Cleaning & Wrangling with Pandas

**Dataset: Malaysia Customer Transactions 2025**

Real-world data is messy! This comprehensive guide covers essential Pandas techniques for:
1. **Data Quality Assessment**: Finding and understanding data issues
2. **Missing Data**: Detection and handling strategies
3. **Duplicates**: Finding and removing redundant data
4. **Data Validation**: Ensuring data integrity
5. **Data Transformation**: Reshaping and creating new features
6. **String Operations**: Cleaning text data
7. **Combining Data**: Merging and concatenating datasets
8. **Grouping & Aggregation**: Summarizing data effectively

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 2)
sns.set_style('whitegrid')

---
## 1. Loading and Initial Exploration

In [ ]:
# Load the Malaysia Customer Transactions dataset
df = pd.read_csv("dummy_malaysia_customer_transactions_2025.csv")

print("Dataset Shape:", df.shape)
print("\nColumn Names:")
print(df.columns.tolist())
print("\nData Types:")
print(df.dtypes)
print("\nMemory Usage:")
print(df.memory_usage(deep=True))

In [ ]:
# First look at the data
df.head(10)

In [ ]:
# Get statistical summary
df.describe()

In [ ]:
# More detailed info
df.info()

---
## 2. Data Quality Assessment

### 2.1 Checking for Missing Values

This dataset **already has real missing values** in `name`, `region`, `number_of_purchases`, and `total_transaction_amount`.

In [ ]:
# Check missing values in the real dataset
df_dirty = df.copy()

# Check missing values
print("Missing Values Count:")
print(df_dirty.isnull().sum())
print("\nMissing Values Percentage:")
print((df_dirty.isnull().sum() / len(df_dirty) * 100).round(2))

In [ ]:
# Visualize missing data
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Heatmap of missing values
missing_matrix = df_dirty.isnull().astype(int)
axes[0].imshow(missing_matrix.T, cmap='YlOrRd', aspect='auto', interpolation='nearest')
axes[0].set_yticks(range(len(df_dirty.columns)))
axes[0].set_yticklabels(df_dirty.columns)
axes[0].set_xlabel('Row Index')
axes[0].set_title('Missing Data Pattern (Red = Missing)')

# Bar chart of missing counts
missing_counts = df_dirty.isnull().sum()
missing_counts[missing_counts > 0].plot(kind='barh', ax=axes[1], color='coral')
axes[1].set_xlabel('Count of Missing Values')
axes[1].set_title('Missing Values by Column')

plt.tight_layout()
plt.show()

### 2.2 Identifying Complete Cases

In [ ]:
# Rows with any missing value
rows_with_missing = df_dirty[df_dirty.isnull().any(axis=1)]
print(f"Rows with missing data: {len(rows_with_missing)}")
print(f"Complete rows: {len(df_dirty) - len(rows_with_missing)}")

# Show some examples
print("\nExamples of rows with missing data:")
rows_with_missing.head()

---
## 3. Handling Missing Data

### 3.1 Dropping Missing Data

In [ ]:
# Drop rows with ANY missing values
df_dropped_any = df_dirty.dropna()
print(f"Original shape: {df_dirty.shape}")
print(f"After dropna(): {df_dropped_any.shape}")

# Drop rows with ALL values missing
df_dropped_all = df_dirty.dropna(how='all')
print(f"After dropna(how='all'): {df_dropped_all.shape}")

# Drop rows with missing in specific columns
df_dropped_subset = df_dirty.dropna(subset=['number_of_purchases', 'total_transaction_amount'])
print(f"After dropna(subset=['number_of_purchases', 'total_transaction_amount']): {df_dropped_subset.shape}")

# Drop columns with missing values
df_dropped_cols = df_dirty.dropna(axis=1)
print(f"After dropping columns with missing: {df_dropped_cols.shape}")

### 3.2 Filling Missing Data

In [ ]:
# Fill numeric columns with a constant value
df_filled_zero = df_dirty.fillna({'number_of_purchases': 0, 'total_transaction_amount': 0})
print("Filled numerics with 0 - Missing count:", df_filled_zero.isnull().sum().sum())

# Fill with column mean
numeric_cols = df_dirty.select_dtypes(include=[np.number]).columns
df_filled_mean = df_dirty.copy()
df_filled_mean[numeric_cols] = df_filled_mean[numeric_cols].fillna(df_filled_mean[numeric_cols].mean())
print("Filled with mean - Missing count:", df_filled_mean.isnull().sum().sum())

# Fill with column median
df_filled_median = df_dirty.copy()
df_filled_median[numeric_cols] = df_filled_median[numeric_cols].fillna(df_filled_median[numeric_cols].median())
print("Filled with median - Missing count:", df_filled_median.isnull().sum().sum())

# Fill different columns with different strategies
df_custom_fill = df_dirty.copy()
df_custom_fill['number_of_purchases'] = df_custom_fill['number_of_purchases'].fillna(df_custom_fill['number_of_purchases'].mean())
df_custom_fill['total_transaction_amount'] = df_custom_fill['total_transaction_amount'].fillna(df_custom_fill['total_transaction_amount'].median())
df_custom_fill['region'] = df_custom_fill['region'].fillna(df_custom_fill['region'].mode()[0])
print("Custom fill - Missing count:", df_custom_fill.isnull().sum().sum())

### 3.3 Forward Fill and Backward Fill

Useful for sequential data (e.g., customers appear in order — propagate last known name/region).

In [ ]:
# Forward fill (propagate last valid observation forward)
df_ffill = df_dirty.ffill()
print("Forward fill - Missing count:", df_ffill.isnull().sum().sum())

# Backward fill (use next valid observation to fill gap)
df_bfill = df_dirty.bfill()
print("Backward fill - Missing count:", df_bfill.isnull().sum().sum())

# Combine forward and backward fill
df_both_fill = df_dirty.ffill().bfill()
print("Both fills - Missing count:", df_both_fill.isnull().sum().sum())

### 3.4 Interpolation

For numeric columns, interpolation estimates missing values from neighbouring values.

In [ ]:
# Linear interpolation on numeric columns
df_interpolated = df_dirty.copy()
df_interpolated[numeric_cols] = df_interpolated[numeric_cols].interpolate(method='linear')
print("Linear interpolation - Missing count:", df_interpolated.isnull().sum().sum())

# Show example: total_transaction_amount before and after interpolation
sample_col = 'total_transaction_amount'
missing_idx = df_dirty[sample_col].isnull()
if missing_idx.sum() > 0:
    sample_idx = df_dirty[sample_col].isnull().idxmax()
    window = slice(max(0, sample_idx - 2), min(len(df_dirty), sample_idx + 3))
    comparison = pd.DataFrame({
        'Original': df_dirty.loc[window, sample_col],
        'Interpolated': df_interpolated.loc[window, sample_col]
    })
    print(f"\nInterpolation example for {sample_col}:")
    print(comparison)

---
## 4. Handling Duplicates

### 4.1 Detecting Duplicates

In [ ]:
# Add some duplicate rows for demonstration
df_with_dupes = df.copy()
df_with_dupes = pd.concat([df_with_dupes, df.head(10), df.iloc[50:55]], ignore_index=True)

print(f"Original dataset size: {len(df)}")
print(f"With duplicates: {len(df_with_dupes)}")

# Check for duplicates
duplicates = df_with_dupes.duplicated()
print(f"\nNumber of duplicate rows: {duplicates.sum()}")

# Show duplicate rows
print("\nDuplicate rows:")
df_with_dupes[duplicates].head()

In [ ]:
# Check duplicates based on specific columns (a customer-quarter pair should be unique)
dupes_on_key = df_with_dupes.duplicated(subset=['customer_id', 'quarter'])
print(f"Duplicates based on customer_id + quarter: {dupes_on_key.sum()}")

# Keep first occurrence
dupes_keep_first = df_with_dupes.duplicated(keep='first')
print(f"Duplicates (keep first): {dupes_keep_first.sum()}")

# Keep last occurrence
dupes_keep_last = df_with_dupes.duplicated(keep='last')
print(f"Duplicates (keep last): {dupes_keep_last.sum()}")

# Mark all duplicates
dupes_keep_false = df_with_dupes.duplicated(keep=False)
print(f"All duplicate rows (including first): {dupes_keep_false.sum()}")

### 4.2 Removing Duplicates

In [ ]:
# Drop duplicates (keep first)
df_no_dupes = df_with_dupes.drop_duplicates()
print(f"After removing duplicates: {len(df_no_dupes)}")

# Drop duplicates based on customer-quarter key
df_no_dupes_subset = df_with_dupes.drop_duplicates(subset=['customer_id', 'quarter'])
print(f"After removing duplicates on customer_id/quarter: {len(df_no_dupes_subset)}")

# Keep last occurrence
df_keep_last = df_with_dupes.drop_duplicates(keep='last')
print(f"Keeping last duplicate: {len(df_keep_last)}")

---
## 5. Data Validation and Quality Checks

### 5.1 Value Range Validation

In [ ]:
# Check for negative values (shouldn't exist in purchases/amounts)
numeric_cols = df.select_dtypes(include=[np.number]).columns

print("Checking for negative values:")
for col in numeric_cols:
    negative_count = (df[col] < 0).sum()
    if negative_count > 0:
        print(f"{col}: {negative_count} negative values")
    else:
        print(f"{col}: ✓ All non-negative")

# Check for outliers using IQR
def find_outliers_iqr(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = data[(data[column] < lower_bound) | (data[column] > upper_bound)]
    return outliers, lower_bound, upper_bound

print("\nOutlier detection for total_transaction_amount:")
outliers, lower, upper = find_outliers_iqr(df.dropna(subset=['total_transaction_amount']), 'total_transaction_amount')
print(f"IQR Range: [{lower:.2f}, {upper:.2f}]")
print(f"Outliers found: {len(outliers)}")
if len(outliers) > 0:
    print(outliers[['customer_id', 'region', 'quarter', 'total_transaction_amount']].head())

### 5.2 Data Type Validation

In [ ]:
# Check data types
print("Current data types:")
print(df.dtypes)

# Ensure numeric columns are actually numeric
df_validated = df.copy()
for col in numeric_cols:
    df_validated[col] = pd.to_numeric(df_validated[col], errors='coerce')

print("\nAfter validation:")
print(df_validated.dtypes)

### 5.3 Unique Value Counts

In [ ]:
# Check unique values in each column
print("Unique value counts:")
for col in df.columns:
    unique_count = df[col].nunique()
    print(f"{col}: {unique_count} unique values")

# Categorical column distributions
print("\nRegion distribution:")
print(df['region'].value_counts())

print("\nQuarter distribution:")
print(df['quarter'].value_counts().sort_index())

---
## 6. Data Transformation

### 6.1 Creating New Columns

In [ ]:
df_transformed = df.dropna(subset=['number_of_purchases', 'total_transaction_amount']).copy()

# Average spend per purchase
df_transformed['Avg_Per_Purchase'] = (
    df_transformed['total_transaction_amount'] / df_transformed['number_of_purchases'].replace(0, np.nan)
)

# Boolean flag for high-value customer
df_transformed['Is_High_Value'] = df_transformed['total_transaction_amount'] > 10000

# Estimated annual spend
df_transformed['Annual_Estimate'] = df_transformed['total_transaction_amount'] * 4

# Revenue contribution percentage per quarter
total_per_quarter = df_transformed.groupby('quarter')['total_transaction_amount'].transform('sum')
df_transformed['Revenue_Share_Pct'] = (df_transformed['total_transaction_amount'] / total_per_quarter) * 100

print("New columns created:")
print(df_transformed[['customer_id', 'number_of_purchases', 'total_transaction_amount',
                       'Avg_Per_Purchase', 'Is_High_Value', 'Annual_Estimate']].head())

### 6.2 Binning Continuous Variables

In [ ]:
# Create spend categories
df_transformed['Spend_Category'] = pd.cut(
    df_transformed['total_transaction_amount'],
    bins=[0, 3000, 8000, 15001],
    labels=['Low', 'Medium', 'High']
)

# Create purchase frequency categories
df_transformed['Freq_Group'] = pd.cut(
    df_transformed['number_of_purchases'],
    bins=[-1, 4, 14, 25],
    labels=['Infrequent', 'Regular', 'Frequent']
)

# Equal-frequency binning (quantiles)
df_transformed['Amount_Quartile'] = pd.qcut(
    df_transformed['total_transaction_amount'], q=4, labels=['Q1', 'Q2', 'Q3', 'Q4']
)

print("Binning results:")
print(df_transformed[['total_transaction_amount', 'Spend_Category',
                       'number_of_purchases', 'Freq_Group', 'Amount_Quartile']].head(10))

print("\nSpend Category distribution:")
print(df_transformed['Spend_Category'].value_counts())

### 6.3 Apply Custom Functions

In [ ]:
# Apply function using entire row (axis=1)
def classify_customer(row):
    if row['total_transaction_amount'] > 12000:
        return 'VIP'
    elif row['number_of_purchases'] > 15:
        return 'Frequent'
    else:
        return 'Standard'

df_transformed['Customer_Class'] = df_transformed.apply(classify_customer, axis=1)

# Lambda functions
df_transformed['Big_Spender'] = df_transformed['total_transaction_amount'].apply(
    lambda x: 'Yes' if x > 10000 else 'No'
)

# Map function: map quarter codes to readable labels
quarter_map = {'Q1-2025': 'Jan-Mar', 'Q2-2025': 'Apr-Jun',
               'Q3-2025': 'Jul-Sep', 'Q4-2025': 'Oct-Dec'}
df_transformed['Quarter_Label'] = df_transformed['quarter'].map(quarter_map)

print("Custom transformations:")
print(df_transformed[['total_transaction_amount', 'number_of_purchases',
                       'Customer_Class', 'Big_Spender', 'Quarter_Label']].head(10))

print("\nCustomer Class distribution:")
print(df_transformed['Customer_Class'].value_counts())

---
## 7. Grouping and Aggregation

### 7.1 Basic GroupBy Operations

In [ ]:
# Group by region and calculate mean transaction amount
region_avg = df_transformed.groupby('region')['total_transaction_amount'].mean().sort_values(ascending=False)
print("Mean transaction amount by region:")
print(region_avg.round(2))

# Multiple aggregations
quarter_stats = df_transformed.groupby('quarter')['total_transaction_amount'].agg(
    ['count', 'mean', 'std', 'min', 'max']
)
print("\nTransaction statistics by quarter:")
print(quarter_stats.round(2))

In [ ]:
# Group by categorical variable
category_stats = df_transformed.groupby('Spend_Category').agg({
    'total_transaction_amount': ['mean', 'std', 'count'],
    'number_of_purchases': 'mean'
})
print("Statistics by Spend Category:")
print(category_stats.round(2))

### 7.2 Advanced Aggregations

In [ ]:
# Multiple groupby columns
region_quarter_stats = df_transformed.groupby(['region', 'quarter'])['total_transaction_amount'].agg(
    ['count', 'mean', 'std']
).round(2)
print("Transaction Amount by Region and Quarter:")
print(region_quarter_stats.head(12))

# Custom aggregation function
def coefficient_of_variation(x):
    return (x.std() / x.mean()) * 100 if x.mean() != 0 else 0

cv_stats = df_transformed.groupby('region')['total_transaction_amount'].agg([
    ('Mean', 'mean'),
    ('StdDev', 'std'),
    ('CV%', coefficient_of_variation)
]).round(2)
print("\nCoefficient of Variation by Region:")
print(cv_stats.head(8))

### 7.3 Transform and Filter

In [ ]:
# Transform: add group mean to each row (region average)
df_transformed['Region_Mean_Amount'] = df_transformed.groupby('region')['total_transaction_amount'].transform('mean')
df_transformed['Amount_vs_Region_Mean'] = df_transformed['total_transaction_amount'] - df_transformed['Region_Mean_Amount']

print("Transform example (how each transaction compares to its region average):")
print(df_transformed[['customer_id', 'region', 'total_transaction_amount',
                       'Region_Mean_Amount', 'Amount_vs_Region_Mean']].head(10).round(2))

# Filter: keep only regions with >= 20 transactions in dataset
regions_large = df_transformed.groupby('region').filter(lambda x: len(x) >= 20)
print(f"\nOriginal rows: {len(df_transformed)}")
print(f"After filtering (regions with >= 20 samples): {len(regions_large)}")

---
## 8. Reshaping Data

### 8.1 Pivot Tables

In [ ]:
# Create pivot table: average transaction amount per region x quarter
pivot = df_transformed.pivot_table(
    values='total_transaction_amount',
    index='region',
    columns='quarter',
    aggfunc='mean'
).round(2)

print("Pivot Table - Mean Transaction Amount (Region × Quarter):")
print(pivot)

# Multiple aggregation functions
pivot_multi = df_transformed.pivot_table(
    values='total_transaction_amount',
    index='region',
    columns='quarter',
    aggfunc=['mean', 'count'],
    fill_value=0
).round(2)

print("\nPivot Table with Multiple Aggregations:")
print(pivot_multi.head())

### 8.2 Melting (Wide to Long)

In [ ]:
# Demonstrate melting with summary data
# Create a wide summary: one row per customer, one column per quarter
wide_summary = df.dropna(subset=['total_transaction_amount']).pivot_table(
    values='total_transaction_amount',
    index='customer_id',
    columns='quarter',
    aggfunc='sum',
    fill_value=0
).head(5).reset_index()

print("Wide format (one row per customer, one column per quarter):")
print(wide_summary)

# Melt back to long format
melted = wide_summary.melt(
    id_vars=['customer_id'],
    var_name='quarter',
    value_name='total_transaction_amount'
)

print("\nMelted (Long format):")
print(melted.sort_values(['customer_id', 'quarter']))

---
## 9. Combining Datasets

### 9.1 Concatenation

In [ ]:
# Split data into quarterly chunks
q1_data = df[df['quarter'] == 'Q1-2025'].copy()
q2_data = df[df['quarter'] == 'Q2-2025'].copy()
q3_data = df[df['quarter'] == 'Q3-2025'].copy()

# Concatenate vertically (stack rows)
stacked = pd.concat([q1_data, q2_data, q3_data], axis=0, ignore_index=True)
print(f"Q1: {len(q1_data)}, Q2: {len(q2_data)}, Q3: {len(q3_data)}")
print(f"Stacked vertically: {stacked.shape}")

# Concatenate horizontally
part1 = df[['customer_id', 'name']].head(10)
part2 = df[['region', 'quarter', 'total_transaction_amount']].head(10)
combined = pd.concat([part1, part2], axis=1)
print(f"\nCombined horizontally: {combined.shape}")
print(combined.head())

### 9.2 Merging DataFrames

In [ ]:
# Create sample datasets to demonstrate merging
# Customer profile table
df_profiles = pd.DataFrame({
    'customer_id': ['CUST00001', 'CUST00002', 'CUST00003', 'CUST00004'],
    'membership_tier': ['Gold', 'Silver', 'Gold', 'Bronze'],
    'signup_year': [2020, 2021, 2019, 2022]
})

# Transaction summary table
df_summary = pd.DataFrame({
    'customer_id': ['CUST00001', 'CUST00002', 'CUST00003', 'CUST00005'],
    'total_annual_spend': [35000, 22000, 41000, 15000]
})

print("Customer Profiles:")
print(df_profiles)
print("\nTransaction Summary:")
print(df_summary)

# Inner join (only matching customer_ids)
inner = pd.merge(df_profiles, df_summary, on='customer_id', how='inner')
print("\nInner Join (matching customers only):")
print(inner)

# Left join (all profiles, fill missing spend with NaN)
left = pd.merge(df_profiles, df_summary, on='customer_id', how='left')
print("\nLeft Join (all profiles):")
print(left)

# Outer join (all from both)
outer = pd.merge(df_profiles, df_summary, on='customer_id', how='outer')
print("\nOuter Join (all customers):")
print(outer)

---
## 10. String Operations

### 10.1 Working with Text Data

In [ ]:
# Work on the customer_id and name columns
df_text = df.dropna(subset=['name']).head(20).copy()

# Create a descriptive ID string
df_text['Transaction_Code'] = df_text.apply(
    lambda x: f"{x['customer_id']}_{x['quarter'].replace('-', '_')}",
    axis=1
)

print("Sample Transaction Codes:")
print(df_text['Transaction_Code'].head(10))

# Extract customer number from customer_id
df_text['Customer_Number'] = df_text['customer_id'].str.extract(r'(\d+)')[0]

# Extract first name
df_text['First_Name'] = df_text['name'].str.split().str[0]

print("\nExtracted information:")
print(df_text[['customer_id', 'Customer_Number', 'name', 'First_Name']].head())

In [ ]:
# String operations
df_text['ID_Lower'] = df_text['customer_id'].str.lower()
df_text['ID_Upper'] = df_text['customer_id'].str.upper()
df_text['Contains_001'] = df_text['customer_id'].str.contains('001')
df_text['Starts_CUST'] = df_text['customer_id'].str.startswith('CUST')
df_text['Split_Parts'] = df_text['Transaction_Code'].str.split('_')

print("String operations:")
print(df_text[['customer_id', 'ID_Lower', 'Contains_001', 'Split_Parts']].head())

---
## 11. Sorting and Ranking

### 11.1 Sorting Data

In [ ]:
df_sorted = df.dropna(subset=['total_transaction_amount']).copy()

# Sort by single column
top_spenders = df_sorted.sort_values('total_transaction_amount', ascending=False)
print("Top 5 highest-spending transactions:")
print(top_spenders[['customer_id', 'name', 'region', 'quarter', 'total_transaction_amount']].head())

# Sort by multiple columns
sorted_multi = df_sorted.sort_values(['region', 'total_transaction_amount'], ascending=[True, False])
print("\nSorted by Region (asc) then Amount (desc):")
print(sorted_multi[['region', 'customer_id', 'quarter', 'total_transaction_amount']].head(10))

### 11.2 Ranking

In [ ]:
# Create ranks
df_ranked = df_sorted.copy()
df_ranked['Amount_Rank'] = df_ranked['total_transaction_amount'].rank(ascending=False).astype(int)
df_ranked['Amount_Percentile'] = df_ranked['total_transaction_amount'].rank(pct=True) * 100

# Rank within groups (rank within each region)
df_ranked['Region_Rank'] = df_ranked.groupby('region')['total_transaction_amount'].rank(ascending=False)

print("Ranking examples:")
print(df_ranked[['customer_id', 'region', 'total_transaction_amount',
                  'Amount_Rank', 'Amount_Percentile', 'Region_Rank']].head(15).round(1))

---
## 12. Comprehensive Data Cleaning Exercise

**Task**: Clean and prepare a messy version of the customer transactions dataset:
1. Handle missing values appropriately
2. Remove duplicates
3. Validate data ranges
4. Create meaningful features
5. Categorize data
6. Produce summary statistics

In [ ]:
# Create messy dataset from the real one
np.random.seed(42)
df_messy = df.copy()

# Add extra missing values
for col in ['number_of_purchases', 'total_transaction_amount']:
    mask = np.random.random(len(df_messy)) < 0.06
    df_messy.loc[mask, col] = np.nan

# Add duplicates
df_messy = pd.concat([df_messy, df_messy.sample(20)], ignore_index=True)

# Add some errors (negative amounts — invalid)
df_messy.loc[df_messy.sample(5).index, 'total_transaction_amount'] = -999

print("Messy dataset created:")
print(f"Shape: {df_messy.shape}")
print(f"Missing values: {df_messy.isnull().sum().sum()}")
print(f"Duplicates: {df_messy.duplicated().sum()}")

In [ ]:
# SOLUTION

# Step 1: Handle missing values
df_clean = df_messy.copy()
print("Step 1: Handling missing values")
print(f"Before: {df_clean.isnull().sum().sum()} missing values")

df_clean['number_of_purchases'] = df_clean['number_of_purchases'].fillna(
    df_clean['number_of_purchases'].median()
)
df_clean['total_transaction_amount'] = df_clean['total_transaction_amount'].fillna(
    df_clean['total_transaction_amount'].median()
)
df_clean['name'] = df_clean['name'].ffill()
df_clean['region'] = df_clean['region'].ffill()

print(f"After: {df_clean.isnull().sum().sum()} missing values")

# Step 2: Remove duplicates
print(f"\nStep 2: Removing duplicates")
print(f"Before: {len(df_clean)} rows")
df_clean = df_clean.drop_duplicates()
print(f"After: {len(df_clean)} rows")

# Step 3: Validate data ranges
print(f"\nStep 3: Validating data ranges")
print(f"Invalid amount values (negative): {(df_clean['total_transaction_amount'] < 0).sum()}")
df_clean = df_clean[df_clean['total_transaction_amount'] >= 0]
df_clean = df_clean[df_clean['number_of_purchases'] >= 0]
print(f"After validation: {len(df_clean)} rows")

# Step 4: Create features
print(f"\nStep 4: Creating features")
df_clean['Avg_Per_Purchase'] = (
    df_clean['total_transaction_amount'] / df_clean['number_of_purchases'].replace(0, np.nan)
)
df_clean['Annual_Estimate'] = df_clean['total_transaction_amount'] * 4
print("Features created: Avg_Per_Purchase, Annual_Estimate")

# Step 5: Categorize
print(f"\nStep 5: Categorizing data")
df_clean['Spend_Category'] = pd.cut(
    df_clean['total_transaction_amount'],
    bins=[0, 3000, 8000, 15001],
    labels=['Low', 'Medium', 'High']
)
df_clean['Customer_Tier'] = df_clean['total_transaction_amount'].apply(
    lambda x: 'VIP' if x > 12000 else ('Regular' if x >= 5000 else 'Occasional')
)
print("Categories created: Spend_Category, Customer_Tier")

# Step 6: Summary statistics
print(f"\nStep 6: Summary Statistics")
print("\n=== Final Cleaned Dataset ===")
print(f"Shape: {df_clean.shape}")
print(f"Missing values: {df_clean.isnull().sum().sum()}")
print(f"Duplicates: {df_clean.duplicated().sum()}")

print("\nMean transaction amount by region:")
print(df_clean.groupby('region')['total_transaction_amount'].agg(['count', 'mean', 'std']).round(2))

print("\nCustomer Tier distribution:")
print(df_clean['Customer_Tier'].value_counts())

print("\n✅ Data cleaning complete!")
df_clean.head()

---
## Summary

### Key Pandas Techniques Covered:

**Data Quality**
- ✅ Missing data detection and visualization
- ✅ Duplicate identification and removal
- ✅ Data validation and quality checks

**Data Handling**
- ✅ Multiple strategies for handling missing data (drop, fill, interpolate)
- ✅ Forward fill, backward fill methods
- ✅ String operations and text processing

**Data Transformation**
- ✅ Creating derived features (Avg_Per_Purchase, Annual_Estimate, Is_High_Value)
- ✅ Binning and categorization (Spend_Category, Freq_Group)
- ✅ Applying custom functions (classify_customer, Customer_Tier)
- ✅ Sorting and ranking

**Data Combination**
- ✅ Concatenating quarterly datasets
- ✅ Merging with customer profiles (inner, left, outer joins)
- ✅ Reshaping (pivot region×quarter, melt wide→long)

**Data Aggregation**
- ✅ GroupBy by region and quarter
- ✅ Multiple aggregation functions
- ✅ Transform (region mean) and filter on groups
- ✅ Pivot tables

### Best Practices:
1. Always inspect your data first (`head()`, `info()`, `describe()`)
2. Check for missing values and duplicates
3. Validate data ranges and types
4. Work on a copy to preserve original data
5. Document your cleaning decisions
6. Create meaningful derived features
7. Use appropriate aggregation methods